## Estimating the marginal effect of NO~2~ on ED Visits using CTMLE

I want to know the marginal effect of NO~2~ > 19 on I have high-dimension data. I was peforming targeted maximum likelihood estimate in R using the TMLE package, but I was having trouble finding covariates to adjust for and results from TMLE due to ETA violations.

Enter Collaborative TMLE (CTMLE) that is specifically designed to tackle this problem. Unforunately there is no R package to perform CTMLE, but there is a Julia package. So here I am: using a Julia kernel, in Jupyter notebooks. 

Tutorial here: https://github.com/lendle/TargetedLearning.jl

### Reading in NO2 ED visit dataframe

In [22]:
# call dataframes and csv packages
using DataFrames, CSV

**Loading Package "GLM"**

Added glm package using *Pkg.add("GLM")*

In [38]:
using GLM

INFO: Precompiling module GLM.


#### NO2 Dataframe

I think the GLM function needs a dataframe and not an array. Going to create dataframe of no2.

In [129]:
# no2_df
no2_df = CSV.read("./asthma_no2_ctmle.csv")

,edvisit,no2_cut,afra_once,mixhouse_once,ppi_once,ppi_sporadic,ppi_frequent,pctlataian0_once,pctlataian0_sporadic,pctlataian0_frequent,pnfhhgt4_once,pnfhhgt4_sporadic,pnfhhgt4_frequent,kidieengg_once,kidieengg_sporadic,kidieengg_frequent,kidasnengn_once,kidasnengn_sporadic,kidasnengn_frequent,kidolengg_once,kidolengg_sporadic,adieengn_once,adieengn_sporadic,adieengn_frequent,srspanengn_once,srspanengn_sporadic,srspanengn_frequent,srieengg_once,srieengg_sporadic,srieengg_frequent,pctpihhd_once,pctpihhd_sporadic,pctpihhd_frequent,pctaiown_once,pctaiown_frequent,pctairent_once,pctairent_sporadic,pctairent_frequent,pctpirent_once,pctpirent_sporadic,pctwood_once,pctwood_sporadic,pctwood_frequent,pownnoph_once,pownnoph_sporadic,pownnoph_frequent,asstct_once,asstct_frequent,x_detres_once,x_commcomr_once,blackmaj_once,majgroup_once
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1
2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1
5,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1
6,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0
7,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1
8,0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
9,0,0,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
10,0,0,1,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


### Logistic Model

Running a logistic model of the crude association between NO2 > 19 and an ED visit. Results are the same as R.

In [130]:
no2_mod = glm(@formula(edvisit ~ no2_cut), no2_df, Binomial(), LogitLink())

StatsModels.DataFrameRegressionModel{GLM.GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Distributions.Binomial{Float64},GLM.LogitLink},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Formula: edvisit ~ 1 + no2_cut

Coefficients:
             Estimate Std.Error  z value Pr(>|z|)
(Intercept)   -1.1995 0.0910428 -13.1752   <1e-38
no2_cut      0.569271  0.213696  2.66393   0.0077


### Set Outcome, Exposure, and Covariate Matrix to Arrays

CTMLE can't use julia dataframes; requires arrays/matrices. Converting dataframe to array.

In [132]:
# edvisit array
edvisit = convert(Array{Float64}, no2_df[:edvisit])
# no2_cut array
no2_cut = convert(Array{Float64}, no2_df[:no2_cut])
# covariate vector
cov_array = Array(no2_df[2:end])

796×51 Array{Union{Int64, Missings.Missing},2}:
 0  0  0  0  0  0  0  0  0  0  0  0  1  …  0  0  0  0  0  0  0  0  1  1  0  1
 0  0  1  0  0  0  0  0  0  0  0  0  1     0  0  0  0  0  0  1  1  1  1  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  1     0  0  0  0  0  0  1  1  1  1  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  1     0  0  0  0  0  0  1  1  1  1  0  1
 0  0  1  0  0  0  0  0  0  0  0  0  1     0  0  0  0  0  0  1  1  1  1  0  1
 0  1  1  0  0  0  0  0  0  0  0  0  1  …  0  0  0  0  0  0  1  1  1  1  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  1     0  0  0  0  0  0  1  1  1  1  0  1
 0  0  1  0  0  0  1  1  1  0  0  0  1     0  0  0  0  0  0  1  0  0  0  0  0
 0  1  1  0  0  0  1  1  1  0  0  0  1     0  0  0  0  0  0  1  0  0  0  0  0
 0  1  1  0  0  0  1  1  1  0  0  0  1     0  0  0  0  0  0  1  0  0  0  0  0
 0  1  1  0  0  0  1  1  1  0  0  0  1  …  0  0  0  0  0  0  1  0  0  0  0  0
 0  1  1  0  0  0  1  1  1  0  1  0  1     0  1  0  0  0  0  0  0  0  0  1  1
 0  1  1  0  0  

## TMLE

Standard TMLE. I think with julia, you first need to set the baseline Q model.

In [144]:
using TargetedLearning

I think TMLE in julia needs to have the model matrix. Running reduced model dataframe.

In [134]:
# define simple model to start
mf = ModelFrame(@formula(no2_cut ~ kidieengg_sporadic + pnfhhgt4_once + afra_once), no2_df)

StatsModels.ModelFrame(796×4 DataFrames.DataFrame
│ Row │ no2_cut │ kidieengg_sporadic │ pnfhhgt4_once │ afra_once │
├─────┼─────────┼────────────────────┼───────────────┼───────────┤
│ 1   │ 0       │ 1                  │ 0             │ 0         │
│ 2   │ 0       │ 1                  │ 0             │ 0         │
│ 3   │ 0       │ 1                  │ 0             │ 0         │
│ 4   │ 0       │ 1                  │ 0             │ 0         │
│ 5   │ 0       │ 1                  │ 0             │ 0         │
│ 6   │ 0       │ 1                  │ 0             │ 1         │
│ 7   │ 0       │ 1                  │ 0             │ 0         │
│ 8   │ 0       │ 1                  │ 0             │ 0         │
│ 9   │ 0       │ 1                  │ 0             │ 1         │
│ 10  │ 0       │ 1                  │ 0             │ 1         │
│ 11  │ 0       │ 1                  │ 0             │ 1         │
⋮
│ 785 │ 0       │ 0                  │ 0             │ 1         │
│ 786 │ 0 

I need to convert to matrix of model I think.

In [135]:
expanded_base_mat = ModelMatrix(mf)

StatsModels.ModelMatrix{Array{Float64,2}}([1.0 1.0 0.0 0.0; 1.0 1.0 0.0 0.0; … ; 1.0 0.0 0.0 1.0; 1.0 0.0 0.0 0.0], [0, 1, 2, 3])

In [147]:
# n obs
#n = size(expanded_base_mat, 1)
# Estimate Q 
#Qfit = glm(@formula(edvisit ~ mf)




LoadError: [91mUndefVarError: ctmle not defined[39m

Couldn't figure this out quite yet. I'll come back to it. I found something that works in R for my data set size.